In [1]:
import pandas as pd
import numpy as np

# !pip install deepctr[gpu]

# 导入deepctr
from deepctr.models import DIN,DIEN,BST
from deepctr.feature_column import SparseFeat, VarLenSparseFeat, DenseFeat, get_feature_names
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras import backend as K
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import * 
import tensorflow as tf

from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler


# import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "2"

generate='./data/generate/'
path='./data/rec_data/'

2022-05-28 18:15:31.853818: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
user_seq=pd.read_csv(generate+'user_seq.csv',usecols=['suv','itemId'])
train=pd.read_csv(path+'train-dataset.csv')
test=pd.read_csv(path+'rec-train-dataset.csv')
test['label']=9
category=pd.read_csv(generate+'category14.csv')
category.drop_duplicates(inplace=True)
train=pd.concat([train,test])
del test
train=pd.concat([train,user_seq])
train=pd.merge(train,category,how='left',on='itemId')

In [3]:
# 数据处理sparse、dense
for feat in ['itemId']:
    lbe = LabelEncoder()
    train[feat] = lbe.fit_transform(train[feat])
    
train[['itemId']]+=1
train[['category']]+=1
    
ygq=train.copy()
user_seq=train[train['sampleId'].isna()]
user_seq=user_seq[['suv','itemId','category']]
train=train[train['sampleId'].notna()]

In [4]:
# 造点统计特征
train['pvid_count'] = train.groupby('pvId')['itemId'].transform('count')
train['pvid_item_nunique'] = train.groupby('pvId')['itemId'].transform('nunique')
train['pvid_suv_nunique'] = train.groupby('pvId')['suv'].transform('nunique')
train['item_count'] = train.groupby('itemId')['itemId'].transform('count')
train['item_suv_nunique'] = train.groupby('pvId')['suv'].transform('nunique')

In [5]:
# train=train[train['userSeq'].notna()]
del train['userSeq']
del train['logTs']

In [6]:
hist_click=user_seq[['suv','itemId','category']].groupby(['suv']).agg({list}).reset_index()
his_behavior_df=pd.DataFrame()
his_behavior_df['suv']=hist_click['suv']
his_behavior_df['hist_itemId']=hist_click['itemId']
his_behavior_df['hist_category']=hist_click['category']
del hist_click
del user_seq

In [7]:
data=pd.merge(train,his_behavior_df,how='left',on='suv')

In [8]:
tmps=data['hist_itemId'].values.tolist()
for i in range(len(tmps)):
    if not isinstance(tmps[i],list):
        tmps[i]=[0]
data['hist_itemId']=tmps

tmps=data['hist_category'].values.tolist()
for i in range(len(tmps)):
    if not isinstance(tmps[i],list):
        tmps[i]=[0]
data['hist_category']=tmps

# data['length'].fillna(0,inplace=True)

del data['sampleId']

In [9]:
# 把特征分开
sparse_fea = ['suv', 'pvId','itemId', 'operator','category',
              'browserType', 'deviceType', 'osType', 'province', 'city']

behavior_fea = ['itemId','category']

hist_behavior_fea = ['hist_itemId','hist_category']

dense_fea = ['pvid_count', 'pvid_item_nunique', 'pvid_suv_nunique', 'item_count', 'item_suv_nunique']
# dense_fea = ['pvid_count', 'pvid_item_nunique', 'pvid_suv_nunique', 'item_suv_nunique']

In [10]:
sparse_fea_tmp = ['suv', 'pvId', 'operator',
              'browserType', 'deviceType', 'osType', 'province', 'city']

# 数据处理sparse、dense
for feat in sparse_fea_tmp:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])

data[dense_fea] = data[dense_fea].fillna(0, )
mms = MinMaxScaler(feature_range=(0, 1))
data[dense_fea] = mms.fit_transform(data[dense_fea])

In [11]:
data[data['label']==9]

,label,pvId,suv,itemId,operator,browserType,deviceType,osType,province,city,category,pvid_count,pvid_item_nunique,pvid_suv_nunique,item_count,item_suv_nunique,hist_itemId,hist_category
4233804,9.0,6,325431,119878,267,27,1,10,33,188,9,0.041667,0.045016,0.0,0.087105,0.0,[0],[0]
4233805,9.0,6,325431,122049,267,27,1,10,33,188,12,0.041667,0.045016,0.0,0.386258,0.0,[0],[0]
4233806,9.0,6,325431,123015,267,27,1,10,33,188,12,0.041667,0.045016,0.0,0.171289,0.0,[0],[0]
4233807,9.0,6,325431,105010,267,27,1,10,33,188,12,0.041667,0.045016,0.0,0.423353,0.0,[0],[0]
4233808,9.0,6,325431,123783,267,27,1,10,33,188,12,0.041667,0.045016,0.0,0.208711,0.0,[0],[0]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5632818,9.0,400025,172922,123707,270,10,1,10,33,120,14,0.041667,0.045016,0.0,0.006647,0.0,"[124040, 126273, 104151, 121979, 110687, 12513...","[3, 9, 9, 10, 9, 9, 9, 9, 9, 15, 9, 9, 9, 9, 9..."
5632819,9.0,400025,172922,126270,270,10,1,10,33,120,12,0.041667,0.045016,0.0,0.078541,0.0,"[124040, 126273, 104151, 121979, 110687, 12513...","[3, 9, 9, 10, 9, 9, 9, 9, 9, 15, 9, 9, 9, 9, 9..."
5632820,9.0,400025,172922,123284,270,10,1,10,33,120,13,0.041667,0.045016,0.0,0.143725,0.0,"[124040, 126273, 104151, 121979, 110687, 12513...","[3, 9, 9, 10, 9, 9, 9, 9, 9, 15, 9, 9, 9, 9, 9..."
5632821,9.0,400025,172922,122306,270,10,1,10,33,120,9,0.041667,0.045016,0.0,0.006493,0.0,"[124040, 126273, 104151, 121979, 110687, 12513...","[3, 9, 9, 10, 9, 9, 9, 9, 9, 15, 9, 9, 9, 9, 9..."


In [12]:
data.to_feather(generate+'DIN_data.feather')
ygq.to_feather(generate+'DIN_data_ygq.feather')

========================================================================

In [2]:
import pandas as pd
import numpy as np

# !pip install deepctr[gpu]

# 导入deepctr
from deepctr.models import DIN,DIEN,BST
from deepctr.feature_column import SparseFeat, VarLenSparseFeat, DenseFeat, get_feature_names
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras import backend as K
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *
import tensorflow as tf

from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler


# import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "2"

generate='./data/generate/'
path='./data/rec_data/'

2022-05-29 00:24:47.636309: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [ ]:
# 把特征分开
sparse_fea = ['suv', 'pvId','itemId', 'operator','category',
              'browserType', 'deviceType', 'osType', 'province', 'city']

behavior_fea = ['itemId','category']

hist_behavior_fea = ['hist_itemId','hist_category']

dense_fea = ['pvid_count', 'pvid_item_nunique', 'pvid_suv_nunique', 'item_count', 'item_suv_nunique']

In [ ]:
data=pd.read_feather(generate+'DIN_data.feather')
ygq=pd.read_feather(generate+'DIN_data_ygq.feather')

In [ ]:
emb_dim=32
max_len=32

sparse_feature_columns = [SparseFeat(feat, vocabulary_size=data[feat].nunique() + 1, embedding_dim=emb_dim) for feat in sparse_fea]
dense_feature_columns = [DenseFeat(feat, 1, ) for feat in dense_fea]

var_feature_columns = [VarLenSparseFeat(SparseFeat(feat, vocabulary_size=ygq[feat.split('_')[1]].nunique() + 1,
                        embedding_dim=emb_dim, embedding_name=feat.split('_')[1]), maxlen=max_len) for feat in hist_behavior_fea]

dnn_feature_columns = sparse_feature_columns + dense_feature_columns + var_feature_columns


In [ ]:
train=data[data['label']!=9]
test=data[data['label']==9]
del data
del ygq
test.head(20)

,label,pvId,suv,itemId,operator,browserType,deviceType,osType,province,city,...,itemId_browserType_nunique,browserType_itemId_nunique,itemId_deviceType_nunique,deviceType_itemId_nunique,itemId_osType_nunique,osType_itemId_nunique,itemId_province_nunique,province_itemId_nunique,itemId_city_nunique,city_itemId_nunique
4233804,9.0,6,325431,119878,267,27,1,10,33,188,...,22,1911,4,3516,7,3358,35,1556,308,1023
4233805,9.0,6,325431,122049,267,27,1,10,33,188,...,25,1911,4,3516,8,3358,36,1556,331,1023
4233806,9.0,6,325431,123015,267,27,1,10,33,188,...,23,1911,4,3516,7,3358,36,1556,311,1023
4233807,9.0,6,325431,105010,267,27,1,10,33,188,...,24,1911,4,3516,7,3358,36,1556,331,1023
4233808,9.0,6,325431,123783,267,27,1,10,33,188,...,23,1911,4,3516,7,3358,36,1556,322,1023
4233809,9.0,6,325431,109509,267,27,1,10,33,188,...,26,1911,4,3516,7,3358,36,1556,332,1023
4233810,9.0,6,325431,122362,267,27,1,10,33,188,...,27,1911,4,3516,8,3358,36,1556,334,1023
4233811,9.0,6,325431,116553,267,27,1,10,33,188,...,27,1911,4,3516,9,3358,36,1556,335,1023
4233812,9.0,6,325431,119117,267,27,1,10,33,188,...,22,1911,4,3516,7,3358,35,1556,306,1023
4233813,9.0,6,325431,115665,267,27,1,10,33,188,...,19,1911,4,3516,7,3358,34,1556,296,1023


In [ ]:
x_trn = {}
for name in get_feature_names(dnn_feature_columns):
    if name in hist_behavior_fea:
        # 这是历史行为序列
        his_list = [l for l in train[name]]
        x_trn[name] = pad_sequences(his_list, maxlen=max_len, padding='post')      # 二维数组
    else:
        x_trn[name] = train[name].values

x_tst = {}
for name in get_feature_names(dnn_feature_columns):
    if name in hist_behavior_fea:
        # 这是历史行为序列
        his_list = [l for l in test[name]]
        x_tst[name] = pad_sequences(his_list, maxlen=max_len, padding='post')      # 二维数组
    else:
        x_tst[name] = test[name].values

In [ ]:
y_trn=train['label'].values
y_tst=test['label'].values
# y_val=val['label'].values

In [ ]:
# 建立模型
model = DIN(dnn_feature_columns, behavior_fea)

# 查看模型结构
model.summary()

# 模型编译
model.compile('adam', 'binary_crossentropy',metrics=['binary_crossentropy','accuracy'])

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
suv (InputLayer)                [(None, 1)]          0                                            
__________________________________________________________________________________________________
pvId (InputLayer)               [(None, 1)]          0                                            
__________________________________________________________________________________________________
itemId (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
operator (InputLayer)           [(None, 1)]          0                                            
______________________________________________________________________________________________

In [ ]:
# history = model.fit(x_trn, y_trn, verbose=1,epochs=3,validation_split=0.2,batch_size=256)
history = model.fit(x_trn, y_trn, verbose=1,epochs=1,batch_size=256)

Train on 4233804 samples


2022-05-29 00:25:36.782021: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-05-29 00:25:36.813201: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1666] Found device 0 with properties: 
name: NVIDIA RTX A5000 major: 8 minor: 6 memoryClockRate(GHz): 1.695
pciBusID: 0000:01:00.0
2022-05-29 00:25:36.813227: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-05-29 00:25:36.824187: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-05-29 00:25:36.827624: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcufft.so.10
2022-05-29 00:25:36.828661: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcurand.so.10
2022-05-29 00:25:36.829408: I tensorflow/stream_executor/platform/defaul

4233804/4233804 [==============================] - 282s 67us/sample - loss: 2.9821 - binary_crossentropy: 2.9821 - acc: 0.8067


In [ ]:
# 模型预测
test['rate'] = model.predict(x_tst, verbose=1, batch_size=256)
# test[['suv', 'itemId', 'rate']].to_csv('./din_rank_score.csv', index=False)

   3328/1399019 [..............................] - ETA: 1:53

2022-05-29 00:30:20.917679: E tensorflow/core/grappler/optimizers/dependency_optimizer.cc:697] Iteration = 0, topological sort failed with message: The graph couldn't be sorted in topological order.
2022-05-29 00:30:20.919115: E tensorflow/core/grappler/optimizers/dependency_optimizer.cc:697] Iteration = 1, topological sort failed with message: The graph couldn't be sorted in topological order.
2022-05-29 00:30:20.920685: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:568] layout failed: Invalid argument: The graph couldn't be sorted in topological order.
2022-05-29 00:30:20.923414: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:568] model_pruner failed: Invalid argument: MutableGraphView::MutableGraphView error: node 'attention_sequence_pooling_layer/local_activation_unit/concat' has self cycle fanin 'attention_sequence_pooling_layer/local_activation_unit/concat'.
2022-05-29 00:30:20.929876: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:568] remapper failed

1399019/1399019 [==============================] - 39s 28us/sample


In [ ]:
label_raw=pd.read_csv(path+'rec-train-dataset.csv',usecols=['label'])
label_raw=label_raw['label'].tolist()
test['label']=label_raw

In [ ]:
ygq=test[['label','pvId','rate']]

In [ ]:
from sklearn.metrics import roc_auc_score
from collections import defaultdict
from tqdm import tqdm
def uAUC(labels, preds, pvId_list):
    user_pred = defaultdict(lambda: [])
    user_truth = defaultdict(lambda: [])
    for idx, truth in enumerate(labels):
        pvId = pvId_list[idx]
        pred = preds[idx]
        truth = labels[idx]
        user_pred[pvId].append(pred)
        user_truth[pvId].append(truth)
    total_auc = 0.0
    size = 0.0
    
    pvId_flag = defaultdict(lambda: False)
    for pvId in set(pvId_list):
        truths = user_truth[pvId]
        flag = False
        for i in range(len(truths) - 1):
            if truths[i] != truths[i + 1]:
                flag = True
                break
        pvId_flag[pvId] = flag

    for pvId in tqdm(set(pvId_list)):
        if pvId_flag[pvId]:
            auc = roc_auc_score(np.asarray(user_truth[pvId]), np.asarray(user_pred[pvId]))
            total_auc += auc 
            size += 1.0
    # size = max(size, 1)
    user_auc = float(total_auc)/size
    return user_auc

In [ ]:
print(uAUC(ygq['label'].values.tolist(), ygq['rate'].values.tolist(), ygq['pvId'].values.tolist()))

100%|██████████| 100106/100106 [01:01<00:00, 1627.84it/s]


0.5


In [ ]:
test

,label,pvId,suv,itemId,operator,browserType,deviceType,osType,province,city,...,browserType_itemId_nunique,itemId_deviceType_nunique,deviceType_itemId_nunique,itemId_osType_nunique,osType_itemId_nunique,itemId_province_nunique,province_itemId_nunique,itemId_city_nunique,city_itemId_nunique,rate
4233804,0,6,325431,119878,267,27,1,10,33,188,...,1911,4,3516,7,3358,35,1556,308,1023,0.0
4233805,0,6,325431,122049,267,27,1,10,33,188,...,1911,4,3516,8,3358,36,1556,331,1023,0.0
4233806,1,6,325431,123015,267,27,1,10,33,188,...,1911,4,3516,7,3358,36,1556,311,1023,0.0
4233807,0,6,325431,105010,267,27,1,10,33,188,...,1911,4,3516,7,3358,36,1556,331,1023,0.0
4233808,0,6,325431,123783,267,27,1,10,33,188,...,1911,4,3516,7,3358,36,1556,322,1023,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5632818,0,400025,172922,123707,270,10,1,10,33,120,...,3052,3,3516,6,3358,32,1556,132,838,0.0
5632819,0,400025,172922,126270,270,10,1,10,33,120,...,3052,4,3516,7,3358,34,1556,306,838,0.0
5632820,0,400025,172922,123284,270,10,1,10,33,120,...,3052,4,3516,7,3358,35,1556,310,838,0.0
5632821,0,400025,172922,122306,270,10,1,10,33,120,...,3052,2,3516,6,3358,31,1556,125,838,0.0
